In [1]:
import torch
from torch.utils.data import DataLoader
from dataset.dataset import get_cdiscount_dataset
from model.model import assemble_model, assemble_model_with_classifier
from model.utils import freeze_layers
from trainer.trainer import get_trainer

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# redirect print to file
# import sys
# sys.stdout = open("PyTorch-resnet34-log.txt", "w")

In [2]:
# configuration
config = {
    'train_batch_size': 128, 'val_batch_size': 128,
    'arch': 'resnet50', 'pretrained': True,
    'optimizer': 'SGD', 'learning_rate': 1e-1, 'momentum': 0.9, 'decay_lr_freq': 2e4, 'weight_decay': 1e-4,
    'resume': None,
    'start_epoch': 0, 'epochs': 10,
    'print_freq': 10, 'validate_freq': 2e4, 'save_freq': 1e4,
    'best_val_prec1': 0
}

In [3]:
import torchvision.models as models

# get dataset
print('getting dataset...')
train_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                      images_csv="train_images.csv",
                                      bson_file_path="/mnt/data/cdiscount/train.bson",
                                      with_label=True,
                                      resize=224)
val_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                    images_csv="val_images.csv",
                                    bson_file_path="/mnt/data/cdiscount/train.bson",
                                    with_label=True,
                                    resize=224)

# get data loader
print('getting data loader...')
train_dataloader = DataLoader(train_dataset, batch_size=config['train_batch_size'], shuffle=True, num_workers=1)
val_dataloader = DataLoader(val_dataset, batch_size=config['val_batch_size'], shuffle=True, num_workers=1)

# define model
print("=> using pre-trained model '{}'".format(config['arch']))
model = models.__dict__[config['arch']](pretrained=config['pretrained'])

# model = torch.nn.Sequential(*list(model.children())[:-1])
# model.add_module('classifier', torch.nn.Linear(in_features=2048, out_features=5270))
# model = torch.nn.DataParallel(model).cuda()
# print(model)

freeze_layers(model, 6)

classifier_layer = [
    torch.nn.Linear(in_features=2048, out_features=5270)
]

# classifier_layer = [
#     torch.nn.Linear(in_features=2048, out_features=5270),
# ]

classifier = torch.nn.Sequential(*classifier_layer)

model = assemble_model_with_classifier(model, -1, classifier)
model = torch.nn.DataParallel(model).cuda()
print(model)

getting dataset...
getting data loader...
=> using pre-trained model 'resnet50'
0 freezing Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
1 freezing BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
2 freezing ReLU (inplace)
3 freezing MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
4 freezing Sequential (
  (0): Bottleneck (
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (relu): ReLU (inplace)
    (downsample): Sequential (
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momen

In [4]:
# define loss function (criterion) and optimizer
criterion = torch.nn.CrossEntropyLoss().cuda()

# get trainer
Trainer = get_trainer(train_dataloader, val_dataloader, model, criterion, config)

# Run!
Trainer.run()

start training
Epoch: [0][0/77344]	Time 18.583 (18.583)	Data 2.957 (2.957)	Loss 8.6027 (8.6027)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [0][10/77344]	Time 1.595 (3.123)	Data 0.015 (0.282)	Loss 6.6623 (7.7387)	Prec@1 10.938 (3.622)	Prec@5 21.875 (8.026)
Epoch: [0][20/77344]	Time 1.616 (2.403)	Data 0.018 (0.155)	Loss 6.9686 (7.3212)	Prec@1 9.375 (6.064)	Prec@5 14.844 (12.649)
Epoch: [0][30/77344]	Time 1.602 (2.148)	Data 0.016 (0.110)	Loss 7.4544 (7.1821)	Prec@1 6.250 (6.578)	Prec@5 10.938 (14.113)
Epoch: [0][40/77344]	Time 1.618 (2.019)	Data 0.015 (0.087)	Loss 6.4774 (7.0384)	Prec@1 7.031 (7.203)	Prec@5 19.531 (15.301)
Epoch: [0][50/77344]	Time 1.621 (1.942)	Data 0.016 (0.073)	Loss 6.2141 (6.8828)	Prec@1 13.281 (8.058)	Prec@5 21.094 (16.942)
Epoch: [0][60/77344]	Time 1.629 (1.891)	Data 0.013 (0.063)	Loss 6.4407 (6.7597)	Prec@1 12.500 (8.696)	Prec@5 25.781 (18.276)
Epoch: [0][70/77344]	Time 1.639 (1.855)	Data 0.014 (0.057)	Loss 6.0230 (6.6626)	Prec@1 10.938 (9.177)	Prec@5 25.781 

Process Process-1:
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/weiso/.local/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 40, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/weiso/.local/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 40, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/weiso/data_science_competition/Cdiscount/src/dataset/dataset.py", line 53, in __getitem__
    img = self.transform(img)
  File "/usr/local/lib/python3.5/dist-packages/torchvision/transforms.py", line 34, in __call__
    img = t(img)
  File "/home/weiso/data_science_competition/Cdiscount/src/dataset/dataset.py", line 15, in __call__
    img = transform

KeyboardInterrupt: 